## Part 2: Simple Text Processing - Tokenization, Lemmatization, Word Frequency, Vectorization (20 pts)

Now we will start working on simple text processing using the `SpaCy` package and the same dataset as Part 1. The package should already be included in the `environment.yml`. However, we will also need to download `en_core_web_sm`, an English language text processing model. To do this, while having your `sotu` environment activated, run the following:

```
python -m spacy download en_core_web_sm
```

Now, you should be good to go!

Some important definitions:

- *Token*: a single word or piece of a word
- *Lemma*: the core component of a word, e.g., "complete" is the lemma for "completed" and "completely"
- *Stop Word*: a common word that does not add semantic value, such as "a", "and", "the", etc.
- *Vectorization*: representing a document as a vector where each index in the vector corresponds to a token or word and each entry is the count.

In this section, we will explore the most common tokens and lemmas throughout different slices of the speech data. We will also develop vectorization representations of the speeches. 

 The core steps are:

1. Process speeches using the SpaCy nlp module
2. Analyze Tokens vs Lemmas:
- Create a list of all tokens across all speeches that are not stop words, punctuation, or spaces.
- Create a second list of the lemmas for these same tokens.
- Display the top 25 for each of these and compare.
3. Analyze common word distributions over different years:
- Create a function that takes the dataset and a year as an input and outputs the top n lemmas for that year's speeches
- Compare the top 10 words for 2023 versus 2019
4. Document Vectorization:
- Train a Term Frequency-Inverse Document Frequency (TF-IDF) vectorization model using your processed dataset and scikit learn
- Output the feature vectors 

**Helpful Resources:**
- https://realpython.com/natural-language-processing-spacy-python/
- https://www.statology.org/text-preprocessing-feature-engineering-spacy/ 
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html# 
- https://www.geeksforgeeks.org/nlp/how-to-store-a-tfidfvectorizer-for-future-use-in-scikit-learn/ 



### Processing Speeches with SpaCy

Lets study just speeches from 2000 and onwards to begin with. So, be sure to subset your DataFrame to just these speeches before continuing!

In [1]:
!pip install spacy

In [2]:
import spacy
!python -m spacy download en_core_web_sm
spacy.cli.download("en_core_web_sm")

from tqdm import tqdm
from collections import Counter
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sou = pd.read_csv('data/SOTU.csv')
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 67.0 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
sou

,President,Year,Text,Word Count
0,Joseph R. Biden,2024.0,"\n[Before speaking, the President presented hi...",8003
1,Joseph R. Biden,2023.0,\nThe President. Mr. Speaker——\n[At this point...,8978
2,Joseph R. Biden,2022.0,"\nThe President. Thank you all very, very much...",7539
3,Joseph R. Biden,2021.0,\nThe President. Thank you. Thank you. Thank y...,7734
4,Donald J. Trump,2020.0,\nThe President. Thank you very much. Thank yo...,6169
...,...,...,...,...
241,George Washington,1791.0,\nFellow-Citizens of the Senate and House of R...,2264
242,George Washington,1790.0,\nFellow-Citizens of the Senate and House of R...,1069
243,George Washington,1790.0,\nFellow-Citizens of the Senate and House of R...,1069
244,George Washington,1790.0,\nFellow-Citizens of the Senate and House of R...,1069


In [4]:
# subset the speech dataframe for speeches from 2000 and onwards
sou_new = sou[sou['Year'] >= 2000]
sou_new

,President,Year,Text,Word Count
0,Joseph R. Biden,2024.0,"\n[Before speaking, the President presented hi...",8003
1,Joseph R. Biden,2023.0,\nThe President. Mr. Speaker——\n[At this point...,8978
2,Joseph R. Biden,2022.0,"\nThe President. Thank you all very, very much...",7539
3,Joseph R. Biden,2021.0,\nThe President. Thank you. Thank you. Thank y...,7734
4,Donald J. Trump,2020.0,\nThe President. Thank you very much. Thank yo...,6169
5,Donald J. Trump,2019.0,"\nThe President. Madam Speaker, Mr. Vice Presi...",5519
6,Donald J. Trump,2018.0,"\nThe President. Mr. Speaker, Mr. Vice Preside...",5755
7,Donald J. Trump,2017.0,"\nThank you very much. Mr. Speaker, Mr. Vice P...",4903
8,Barack Obama,2016.0,"\nThank you. Mr. Speaker, Mr. Vice President, ...",5956
9,Barack Obama,2015.0,"\nThe President. Mr. Speaker, Mr. Vice Preside...",6659


In [5]:
# Process each speeches using the 'nlp' function
processed_speeches = []

for i in range(len(sou_new)):
    speech_text = sou_new.loc[i, 'Text'] 
    processed = nlp(speech_text)    
    processed_speeches.append(processed)

### Analyze Tokens vs Lemmas

#### Token List

Create a list of tokens across all speeches that are not spaces, stopwords, or punctuation. Make each token lowercase as well. *Hint: each element of the list we just created are themselves lists of tokens. Token objects have attributes `is_stop`, `is_punct`, and `is_space`.*

In [6]:
tokens = []

for speech in processed_speeches:
    for token in speech:
        if not token.is_punct and not token.is_space and not token.is_stop:
            tokens.append(token.text)
    

In [7]:
# print top 20 tokens
# Hint - use Counter, and one of the Counter object's methods to display the top 20
token_count = Counter(tokens).most_common(20)
print(token_count)

[('America', 816), ('people', 622), ('American', 582), ('new', 495), ('years', 439), ('Americans', 437), ('world', 409), ('year', 401), ('country', 369), ('jobs', 325), ('work', 324), ('know', 323), ('Congress', 317), ('time', 297), ('help', 278), ('need', 266), ('tonight', 253), ('President', 246), ('economy', 243), ('want', 237)]


#### Lemma List

Do the same as above, but for lemmas. *Hint: recall lemmas are components of words. Each token should have an attribute to extract the lemma.*

In [ ]:
lemmas = []

for token in tokens:
    word = nlp(token) # run the tokens through nlp to make sure it has the lemma_ attribute
    for token in word:
        lemma=token.lemma_
        lemmas.append(lemma.lower())

In [ ]:
# print top 20 lemmas
lemma_counts = Counter(lemmas).most_common(20)
lemma_counts

#### Token versus Lemma Comparison

What do you notice about the top tokens versus the top lemmas? 
Consider two tokens - "year" and "years" - how do their counts compare to the lemma "year"?
What about the lemma "child"?

### Common Words

#### Common Words per Year Function

Fill in the below function to obtain the n-most common words in speeches for a given year.

inputs: 
- df raw unprocessed sou dataframe
- year
- n
outputs: 
- top n words for that years

steps:
- subset the dataframe for the year of interest - note the years might not be in int type
- process the subsetted dataframe with spacy
- get the lemmas across all those speeches
- count the top n lemmas

In [ ]:
def get_most_common_words(df, year, n=25):
    """
    Processes the SOTU speech for a given year and returns
    the most common non-stopword/punctuation lemmas.

    INPUTS
    - raw unprocessed sou data frame
    - year

    OUTPUTS
    - top n words for that year
    """

    # Step 1: Subset df
    new_df = df[df['Year'] == year]
    
    # Step 2: Process the text with spaCy
    new_df = df[df['Year'] == year].reset_index(drop=True) 
    processed_speeches = []

    for i in range(len(new_df)):
        speech_text = new_df.loc[i, 'Text'] 
        processed = nlp(speech_text)    
        processed_speeches.append(processed)
    
    # Step 3: Get lemmas
    lemmas = []

    for doc in processed_speeches:
        for token in doc:
            if not token.is_punct and not token.is_space and not token.is_stop:
                lemmas.append(token.lemma_.lower())
    
    lemma_counts = Counter(lemmas).most_common(n)
    
    return lemma_counts

In [ ]:
# test it on 2024
get_most_common_words(sou, 2024, n=20)

#### Compare 2023 to 2017

Run your function from the previous step to get the top 20 words for 2017 and 2023. Plot the words and their frequencies in a barchart and replicate the figure below.

In [ ]:
words_2023 = get_most_common_words(sou, 2023, n=20)
words_2017 = get_most_common_words(sou, 2017, n=20)

In [ ]:
words_2023

In [ ]:
words_2017

In [ ]:
# Hint - put the words and counts into a pd Dataframe for better structure
# and to make plotting easier
df_2017 = pd.DataFrame(words_2017, columns=['lemma', 'count'])
df_2023 = pd.DataFrame(words_2023, columns=['lemma', 'count'])

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 10)) 
# Plot 2017
sns.barplot(
    x = 'lemma',
    y = 'count',
    data = df_2017,
    ax = axes[0],
     color='#2C7BA1'
    
)
axes[0].set_title('2017 State of the Union Most Frequent Words')
axes[0].set_xlabel('Word')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45) 

# Plot 2023
sns.barplot(
    x='lemma',
    y='count',
    data=df_2023,
    ax=axes[1],
    color='#2C7BA1'
)

axes[1].set_title('2023 State of the Union Most Frequent Words')
axes[1].set_xlabel('Word')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()

#saving figure
plt.savefig('outputs/sotu_word_freq2017_2023.png')

### TF-IDF Vectorization

To use statsitical alorithms on documents, we need to transform them into vectors, where each element of the vector corresponds to a particular word in a document or corpus of documents. One common way is via TF-IDF embeddings. LLMs work similarly - they typically use transformer models to generate text embeddings before sending text through a deep neural network. 

Here we will fit a TF-IDF vectorizer, plot all the speeches on a 2-D grid using PCA and also using a heatmap, and examine TF-IDF scores for the top 10 most common words in the first speech. This is a good resource here: https://medium.com/GeoffreyGordonAshbrook/vector-visualization-2d-plot-your-tf-idf-with-pca-83fa9fccb1d 

In [ ]:
# Install nlk to current kernel only once, can do from terminal 
#(has been added to environment.yml too so if using this, do not have to install in notebook)

import sys, subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "nltk"])

# confirm the version that was just installed
import nltk
print("NLTK version:", nltk.__version__)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from nltk.tokenize.casual import casual_tokenize

#### Train the Vectorizer and Transform the Data

In [ ]:
# you may use this as input to fit the TF-IDF vectorizer
raw_docs = sou["Text"].to_list()

In [ ]:
##########################
# Minimal TF-IDF Vectors (Reference: Vector Visualization by Ashbrook)
##########################

# Select Model
tfidf_model = TfidfVectorizer()

# Fit Model
tfidf_vectors = tfidf_model.fit_transform(raw_docs).toarray()

The output of `fit_transform()` will be a matrix where each row corresponds to a speech, each column corresponds to a word in the corpus of speeches, and the value is the TF-IDF score which measures the importance of that word in that speech, relative to the rest of the speeches.

#### Plot Speeches

- First used PCA to generate the first chart
- Second use seaborn heatmap with a log-scaled color axis to generate the second chart

In [ ]:
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
%matplotlib inline
# Step 1: Set PCA to find first 2 principal components
pca = PCA(n_components=2)

# Step 2: Create a new dataframe where each row is a speech, and each column is a projection onto
# one of the two principal components
speeches_array = tfidf_vectors
speeches_df2d = pd.DataFrame(pca.fit_transform(speeches_array), columns=list('xy'))

# Plot Data Visualization (Matplotlib)

# Plot (matplotlib only)
plt.style.use('seaborn-v0_8')           # just a style; still matplotlibplt.figure(figsize=(6,4))
plt.figure(figsize=(6,4))
plt.scatter(speeches_df2d['x'], speeches_df2d['y'], s=20)
plt.xlabel('Principle Component 1'); plt.ylabel('Principle Component 2'); plt.title('Plot of Vectorized Speeches Principle Components')
plt.grid(False)
plt.tight_layout(); plt.show()


In [ ]:
import numpy as np
import seaborn as sns
from matplotlib.colors import LogNorm
import matplotlib.ticker as mticker
from matplotlib.ticker import FixedLocator, FixedFormatter

# vectorized_docs: scipy sparse matrix (rows=speeches, cols=tokens)
A = tfidf_vectors.astype(np.float32)   # densify

# Log scale needs vmin > 0; pick the smallest positive value
pos_min = A[A > 0].min() if (A > 0).any() else 1e-6

plt.style.use('seaborn-v0_8')
plt.figure(figsize=(5.8, 4.7))

ax = sns.heatmap(
    A,
    cmap="magma",
    norm=LogNorm(vmin=pos_min, vmax=A.max()),
    cbar_kws={"format": mticker.LogFormatterMathtext()}
)

ax.set_title("Vectorized Speeches", pad=8)
ax.set_xlabel("Vector Index")
ax.set_ylabel("Speech Index")

n_cols = A.shape[1]
step = 928         
xticks = np.arange(0, n_cols, step)

ax.xaxis.set_major_locator(FixedLocator(xticks))
ax.xaxis.set_major_formatter(FixedFormatter([str(v) for v in xticks]))
plt.setp(ax.get_xticklabels(), rotation=90)
plt.grid(False)
plt.tight_layout()
plt.show()

#### Get the TF-IDF value for certain words and documents

In [ ]:
word_list = ['year',
 'america',
 'people',
 'american',
 'work',
 'new',
 'job',
 'country',
 'americans',
 'world'] # top ten most common words through whole corpus

In [ ]:
word_nums = [tfidf_model.vocabulary_[w] for w in word_list] # get each word's index number using the .vocabular_ attributed of vectorizer

In [ ]:
idf_score = tfidf_model.idf_[word_nums] # get their IDF score by using .idf_ at the indices from the previous step

In [ ]:
tf_idf = tfidf_vectors[0, word_nums] # get the tf_idf score for the first speech

In [ ]:
pd.DataFrame({"Word": word_list, "IDF Score": idf_score, "TF-IDF Score": tf_idf})